In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import json
import re

In [3]:
data = pd.read_csv('/kaggle/input/house-prices/data.csv')

In [4]:
data.head(3)

In [5]:
data.info()

In [6]:
data.columns

* 'status'	    - статус, 
* 'private pool	- частный бассейн, 
* 'propertyType'	- тип недвижимости, поместья, свойства, 
* 'street'	    - улица, 
* 'baths'         - бани,     
* 'homeFacts'	- домашние факты, сведения о доме
* 'fireplace'	- камин, 
* 'city'		- город, 
* 'schools'	- школы, 
* 'sqft'		- квадратный фут, 
* 'zipcode'	- почтовый индекс, 
* 'beds'		- кровати, спальня
* 'state'		- государство, штаты, 
* 'stories'	- рассказы--(этажность недвижимости)?, 
* 'mls-id'	- млс-идентификатор, 
* 'PrivatePool'	- Частный бассейн, 
* 'MlsId'		- MlsId идентификатор,
* 'target'	- цель (цена недвижиости)* 'status'	    - статус.


## EDA

#### Признак "target"

In [7]:
data.isna().sum()/(data.shape[0]/100)

Строки с пустыми значениями признака "target" надо удалять.

In [8]:
data.drop(data[data.target.isna()].index, inplace=True)
data = data.reset_index(drop=True)

In [9]:
def make_target(x):
    x = re.sub('[^0-9]', '', x)
    x = int(x)
    return x

data.target = data.target.apply(make_target)

In [10]:
data.target.describe()

По содержанию будем предполагать, что признаки ['street', 'mls-id', 'MlsId', 'zipcode'] не должны сильно влиять на цену недвижимости. Поэтому удаляем этих признаков.

In [11]:
cols_drop = ['street', 'zipcode',  'mls-id', 'MlsId']
data.drop(cols_drop, axis=1, inplace=True)

In [12]:
df = data.copy()
df.shape

##### Если данные не качественные, то любая хорошая модель не поможет получить  хороший результат
Для анализа данных признаков используются общие стандартные методы python, numpy, pandas, matplotlib, seaborn, модели sklearn, разные приемы парсинга. При необходимости придется анализировать данные построчно.

#### Признак "status"

In [13]:
df['status'].value_counts().index.tolist()

In [14]:
df['status'].nunique()

'Pf' = 'Pre foreclosure'
'Ps' = 'Pending Sale'
'Bckp' = 'Backup'
W/ Cont. = 'with contract' ???
'insp' = 'inspection'

In [15]:
df['status'].value_counts()

In [16]:
def make_status(x):
    if x.startswith('Coming soon'): x = 'coming soon'
    x = x.replace('Showing', 'show')
    x = x.replace('Pf', 'pre foreclosure')
    x = x.replace('Ps', 'pending Sale')
    x = x.replace('Bckp', 'backups')
    # x = x.replace('insp', 'inspection')              
    x = x.lower()
    x = re.sub('[^a-z:]', ' ', x)
    x = re.sub(r'\b\w{,2}\b', '', x)
    x = re.sub(r'\s+', ' ', x)
    return x.strip()

In [17]:
df['status'] = df['status'].fillna('other')
df['status'] = df['status'].apply(make_status)

df['status'].value_counts()

In [18]:
# df['status'].value_counts().index.tolist()

Термины, используемые в сфере продажи-покупки недвижимости:
status - статус, состояние
estate - недвижимость
Multiple Listing Service (MLS) - Служба множественного листинга 

'foreclosed' - 'исключено', 'лишен права выкупа'
'foreclosure' - 'выкупа'. — это процесс, когда ваша недвижимость переходит к кредитору из-за неуплаты Вами ипотечных выплат. Продажа foreclosure недвижимости отличается от традиционных продаж. 
Переход недвижимости в собственность залогодержателя если, по какой-либо причине, вы не можете выплачивать ваш mortgage
Mortgage - ссуда в банке на недвижимость (Ипотека).

'pre foreclosure' - «до выкупа»
'pre foreclosure auction' - «аукцион до выкупа»

'under contract showing',
'under contract show',
'under contract' - 'по договору',
'active under contract' - «действующий по контракту»,

'under contract backups' - резервное копирование по контракту,
'active backup'  -  «активное резервное копирование»
'backup contract' - «резервный контракт»
'active backup' - «активное резервное копирование»

'contingency' - 'непредвиденные обстоятельства'
'contingency contract' - «контракт на случай непредвиденных обстоятельств»,
'active contingency' - 'активное непредвиденное обстоятельство',
'insp inspection contingency' - 'непредвиденные обстоятельства инспектирования'

'pending escape clause' - 'ожидающая оговорка об освобождении'

'pending backup wanted' - «требуется резервное копирование в ожидании»
'pending take backups' - «ожидание создания резервных копий»
 
'pending continue show financing' - 'в ожидании продолжения шоу финансирования'
'pending continue show' - 'ожидание продолжения шоу'

'pending insp finance' - 'в ожидании финансовой поддержки'
'pending inspection' - "в ожидании проверки"

'due diligence period' - «период должной осмотрительности»

'activated' - активирован (active)

'active with contract' - 'активен с контрактом'
'active option' =
= active with contract =
= 'active option contract' =
= contingency period =
= due diligence period
«активный вариант» - «активный опционный контракт»,
In some states, the active option contract is referred to as a contingency period or due diligence period.

'active with offer' - 'активен с предложением'


'active auction',
'auction active',
'auction'

contingency, contingencies - случайность, непредвиденные обстоятельства 
contingent - условный. В сфере недвижимости, когда дом указан как условный, это означает, что предложение было сделано и принято, но до завершения сделки должны быть выполнены некоторые дополнительные критерии. В сфере недвижимости, когда дом указан как условный, это означает, что предложение было сделано и принято, но до завершения сделки должны быть выполнены некоторые дополнительные критерии.

cooling-off period - период обдумывание (период охлаждения)
conditional offers - условные предложения


In [19]:
df['status'] = df['status'].apply(lambda x: 'other' if x=='' else x)

In [20]:
df['status'].value_counts()['continue show']

In [21]:
# Заменим некоторые слова для приобретения единой формы
old_words = ['auction active', 'pending continue show financing', 'activated']
new_words = ['active auction', 'pending continue show','active']

for i in range(0,len(new_words)):
    df['status'] = df['status'].apply(lambda x: 
                                    x.replace(old_words[i], new_words[i]))

In [22]:
new_string = ['active with contract', 'active option contract', 
              'contingency period', 'due diligence period']
for i in range(0, len(new_string)):
    df['status'] = df['status'].apply(lambda x: 'active option' 
                                    if (-1 != x.find(new_string[i])) else x)

df['status'] = df['status'].apply(lambda x: 'for rent' 
                                        if (-1 != x.find('for rent')) else x)

In [23]:
df['status'] = df['status'].apply(lambda x: 'active auction' 
                                  if x=='auction' else x)
df['status'].nunique()

In [24]:
df['status'].value_counts().index.tolist()

In [25]:
# df['status'].value_counts()

Статусы недвижимости разнобразны - 72 вида. По ходу модельного эксперимента рассмотрим варианты уменьшения категории, выбрав топ 50 или 30 категории.

In [26]:
top_status = df['status'].value_counts()[:50].index
df['status'] = df['status'].apply(lambda x: x 
                                              if x in top_status else 'other')

In [27]:
df['status'].value_counts()

#### Признак "propertyType"

Типы домов, жилищ, и их основные характеристики:
1. Apartment - Квартира
2. Condo - Квартира (Кондоминиум)
3. Co-Op - Совместная (Кооперативная квартира)
4. Single-Family (Detached)
5. Tiny Home -  Односемейный (маленький, отдельный)
6.  Townhome, Townhouse 
7. Cape Cod - черепица, деревянный сайдинг, центральная дверь с окнами по бокам, 1-2 этажf.
8. Colonial home - Два этажа с лишним, симметричный, центральная лестница, формальный вид. Different types include English, French, Dutch, Georgian, Spanish and American (which closely resembles English) colonials.
9. Contemporary  - Основные характеристики: Чистые, простые линии; нейтральные цвета; натуральные текстуры 
10. Cottage - Уютная, небольшая веранда; небольшие жилые помещения; индивидуальный характер.
11. Craftsman - "ремесленник"
    элементы ручной работы из дерева, выступающие балки, большие колонны.
12. Greek Revival - 
     большие белые колонны, украшения в греческом стиле, парадный вход.
13. Farmhouse -  
    деревенское, прямоугольное, большое крыльцо, крыши в стиле сарая и особенности.
14. French Country - 
    влияние прованса, каменный внешний вид, состаренный вид.
15. Mediterranean - белая штукатурка, теплое дерево и камень, черепичные крыши, есть элементы испанских и итальянских вилл.
16. Midcentury Modern - 
    сочетание органики и геометрии, гладкий, лаконичный, большие окна.
17. Ranch - 
     один этаж, низкая крыша, открытые жилые помещения, задний двор.
18. Split-Level - 
    Жилые помещения разделены короткими лестничными пролетами.
19. Tudor - 
    Деревянный каркас, каменная кладка, ассимметричные, крутые двускатные крыши.
20. Victorian (Queen Anne Houses) - два-три этажа, богато украшенные крутые
    двускатные крыши, небольшие башни, яркие фасады.

Источник:
https://www.rocketmortgage.com/learn/types-of-houses

Europian Houses style - 
    часто используют кирпич или камень и включают в себя высокие крутые крыши, высокие окна, часто со ставнями, и традиционные декоративные детали, такие как фронтоны и замковые камни.

log home - деревяный дом
    
mortgage - ипотека


Некоторые жаргонные сокращения, для исправления.

'farms/ranches' - 1-2 story
  
less than 4 floors == 3 story
'4+ floors'

* cluster home
* modular
* prefab
* manufactured home (house)  = mobile | prefab | modular
* transitional - внутри домашний стиль

* mo2 le - mobile
* mo2le  - mobile
* 'ca2 n' - cabin

* midcentury
* mid century
* mid-century

In [28]:
df.shape

In [29]:
df['propertyType'].isna().sum()

In [30]:
df['propertyType'] = df['propertyType'].fillna('no data')
df['propertyType'] = df['propertyType'].apply(lambda x: 'no data' 
                                        if x ==' ' or x=='unknown' else x)
df['propertyType'] = df['propertyType'].apply(lambda x: x.lower())

df['propertyType'].value_counts()

In [31]:
df['propertyType'].value_counts()['singlefamilyresidence']

In [32]:
df['propertyType'].value_counts().index.tolist()

In [33]:
df['propertyType'].value_counts()['yes']

Строки с "land" значением признака 'propertyType' удаляем. Так как нам необходимо оценить недвижимость. Земельный участок нас не интересует. 

In [34]:
df['propertyType'] = df['propertyType'].apply(lambda x: x.replace('lot/land', 'land'))
df['propertyType'] = df['propertyType'].apply(lambda x: 'land' if (-1 != x.find('land')) else x)

In [35]:
ind_land_drop = df[df['propertyType']=='land'].index
ind_land_drop

In [36]:
df.drop(ind_land_drop,  inplace=True)
df = df.reset_index(drop=True)

Содержимое признака небрежно заполнено и данные очень разнообразны, и требуют сложного парсинга содежимого признака.
Заменим некоторые сокрашенные слова, расшифруем и заменим синонимы на более общепринятые.

In [37]:
old = ['one', 'two', '2-', 'stories', '1 1/2', 'tri-', 'single family',
       'singlefamilyresidence', 'townhome',   'bi', 'bi-', 'multi family', 
       'hi-', 'high ', 'condominium',  '4 plex', 'other', 'ca2 n', 'cape cod',
       'mid century', 'mid-century',  'mo2 le', 'mo2le', ' / ','mfd',   
       'co-op', 'coop',  'modernist', 'key west', 'farm house',
       'queen anne', ' -level', 'floors', 'mobile home', 'manuf/mobile',
       'less than 4 story', 'historic/older', 'quad']
new = ['1', '2', '2 ', 'story',  '1.5', '3 ', 'single-family',
       'single-family', 'townhouse',  '2 ', '2 ',  'multi-family',
       'high-', 'high-', 'condo',  '4-plex', 'other style', 'cabin', 'cape', 
       'midcentury', 'midcentury', 'mobile', 'mobile', '/', 'manufactured', 
        'cooperative', 'cooperative',  'modern', 'elevated', 'farmhouse',
        'victorian', ' level',  'story', 'manufactured', 'manufactured',
        '3 story', 'historical', '4']
for i in range(0,len(new)):
    df['propertyType'] = df['propertyType'].apply(lambda x: 
                                                  x.replace(old[i], new[i]))

In [38]:
df['propertyType'].value_counts()

In [39]:
# df['propertyType'].value_counts().index.tolist() 

In [40]:
# В признаке 'propertyType' много сведений об этажности домов
# Выделим все  эти данные
# Это небходимо для уточнения и дополнения признака 'stories'
df['Property_story'] = df['propertyType'].apply(lambda x: 
                                        x if -1!= x.find('story') else 'no')

In [41]:
df['Property_story'].value_counts()

In [42]:
# df['Property_story'].value_counts().index.tolist()

In [43]:
# Dыделим индексы строк, имеющие данные об этажности
ind_story = df[df['Property_story']!='no'].index
ind_story

In [44]:
df['stories'][ind_story].value_counts()

In [45]:
df['Property_story'][ind_story].value_counts().index.tolist()

In [46]:
# Оставляем сведения только относящиеся к этажности и записываем в признак 'stories'
def story(x):
    s = x.split(',')
    se = ''
    for i in range(0, len(s)):
        if s[i].find('story')!=-1:
            se = s[i]
    return se.strip()

df.loc[ind_story, 'stories'] = df['Property_story'][ind_story].apply(story)
df['stories'][ind_story].value_counts()

In [47]:
# Очищаем от слов, относящихся к типам домов и записываем в признак 'stories'
del_str = ['/ranch', 'manufactured', 'coastal', 'traditional', 'condo']

def del_substr(x):
    for i in range(0, len(del_str)):
        x = x.replace(del_str[i], '').strip()
    return x

df.loc[ind_story, 'stories'] = df['stories'][ind_story].apply(del_substr)
df['stories'][ind_story].value_counts()

In [48]:
# Удаление сведений, относящихся к этажности для перезаписи в признак 'propertyType'
def type_del_story(x):
    s = x.split(',')
    for i in range(0, len(s)):
        if s[i].find('story')!=-1:
            if len(s)==1 and i==0:
                s[i] = 'other'
            if len(s)>1:
                s[i] = ','
            if len(s)>1 and i==0:
                s[i]=''
        x = '' .join(s).strip()
    return x

In [49]:
df.loc[ind_story, 'propertyType'] = df['Property_story'][ind_story].apply(type_del_story)

df['propertyType'][ind_story].value_counts()

In [50]:
df.drop('Property_story', axis=1, inplace=True)

Далее в полях признака 'propertyType' оставляем основные типы домов, жилищ

In [51]:
# Присвоение полям новых названий в соответствии с найденной подстрокой
new = ['mediterranean', 'english', 'georgian', 'florida','spanish',
        'lake house', 'tudor', 'victorian', 'ranch', 'townhouse', 
        'single-family','multi-family', 'single detached', 'detached', 
        'high-rise', 'colonial', 'traditional',
        'condo', 'contemporary', 'other style', 'bungalow', 'craftsman',
        'cape cod','cooperative', 'a-frame',  'custom', 'loft', 
        'multi-level', 'attached', 'manufactured',  'side-by-side', 
        'cottage',  'french', 'cluster home', 'garden home', 
        'farmhouse', 'elevated',  'historical', 'transitional','penthouse',
        'modern']

for i in range(0, len(new)):
    df['propertyType'] = df['propertyType'].apply(lambda x: new[i] 
                                            if (-1 != x.find(new[i])) else x)

In [52]:
#  Присвоение  полям новых значений с изменением
old = ['cape','log']
new = ['cape cod','log home']

for i in range(0,len(new)):
    df['propertyType'] = df['propertyType'].apply(lambda x: new[i] 
                                              if -1!=x.find(old[i])  else x)
df['propertyType'] = df['propertyType'].apply(lambda x: 'historical'
                        if (x=='historic') or (x=='historic vintage') else x)

In [53]:
df['propertyType'].value_counts()

In [54]:
# df['propertyType'].value_counts().index.tolist() 

Непонятные значения признака 'propertyType':
'no data' - эти данные - NaN
'camp', 'end unit', 'yes' - что эти данные означают для  типов квартир и домов?

Привести к единым данным:

'no data' = 'other style' = 'other' ?!

In [55]:
df['propertyType'] = df['propertyType'].apply(lambda x: 'other style' 
                            if (x=='no data') or (x=='other')  else x)

In [56]:
# С целью уменьшения числа категорий, выберем топ 50 или топ 100 типов стилей домов.
top_Type = df['propertyType'].value_counts()[:50].index
df['propertyType'] = df['propertyType'].apply(lambda x: x 
                                            if x in top_Type else 'other style')

In [57]:
df['propertyType'].value_counts()

#### Признак "stories"

Признаки 'stories' и 'propertyType' необходимо рассмотреь совместно, так как в признаке 'propertyType' приведены данные об этажности домов, которые относятся к признаку 'stories'.

In [58]:
df['stories'].value_counts()

In [59]:
df['stories'].value_counts().index.tolist()

In [60]:
df['stories'].isna().sum()

In [61]:
df['stories'] = df['stories'].fillna('0.0')
df['stories'] = df['stories'].apply(lambda x: x.lower())
df['stories'] = df['stories'].apply(lambda x: '0.0' if x == '0' else x)

In [62]:
num_str = ['one','two','bi', 'split', 'three','tri','four', 'quad' , 'stories', 
           '-level', 'levels', 'leveland','cape cod', '1 and 1 half', '1 1/2 story',
           'ranch', 'garden', 'log home', 'cape',
          'raised ranch', 'bungalow', 'farm house', '4plex', 'sixplex','tudor']
num_num = ['1','2','2', '2', '3','3', '4', '4', 'story', 
           ' level', 'level', 'level and','1 story', '1.5 story', '1.5 story',
           '1 story', '1 story', '1 story', '1 story',
          '1 story', '1 story', '1 story', '4 plex','6 plex', '2 story']
for i in range(0, len(num_str)):
    df['stories'] = df['stories'].apply(lambda x: x.replace(num_str[i], num_num[i]))

'dwelling story' - жилой дом
'1 story/f.r.o.g.'
FROG is a “Finished Room Over Garage.” 

foyer - очаг

t3-townhouse стиль - 2 и более этажей

'craftsman' 1, 1.5 этажей

'log' - 'log home' 1,2 этажей


In [63]:
df['stories'].value_counts()

In [64]:
# df['stories'].value_counts().index.tolist()

Проверим последовательно, признаки  какой этажности соответствуют признаку 'stories'  к данным 'townhouse', 'condominium', 'traditional' и 'Contemporary' из признака 'propertyType' в исходном датасете (dataframe data). В исходном датасете они еще не обработаны, т.е. в виде 'Townhouse', 'Condominium', 'Traditional', 'Contemporary'.

In [65]:
# 'Townhouse', 'Condominium', 'Traditional', 'Contemporary'
data['stories'][data['propertyType']=='Townhouse'].value_counts().index[0]

In [66]:
data['stories'][data['propertyType']=='Condominium'].value_counts().index[0]

In [67]:
data['stories'][data['propertyType']=='Traditional'].value_counts().index[0]

In [68]:
data['stories'][data['propertyType']=='Contemporary'].value_counts().index[0]

In [69]:
df['stories'].value_counts()['ca2n/cottage']

In [70]:
# df['stories'].value_counts().index.tolist()

In [71]:
one_story = ['condominium', 'traditional', 'contemporary', 
             'mo2le', 'ground level', 'modular home', 'craftsman']
def change_story(x):
    if x=='townhouse':
        x = '2'
    if x in one_story:
        x = '1'
    if -1!=x.find('low-rise') or -1!=x.find('multi level'):
        x = '2'
    if -1!=x.find('mid-rise'):
        x = '5'
    if -1!=x.find('high-rise'):
        x = '9'
    return x

df['stories'] = df['stories'].apply(change_story)  

In [72]:
df['stories'].value_counts().index.tolist()

In [73]:
def stories_float(x):
    if x.startswith(','): x = x[1:]
    if x == -1.0: return x
    x = re.sub('[^0-9,-\.]', '', x)
    x = re.sub('-',',', x)
    x = x.replace(',,',',0,')
    x = x.split(',')
    try:
        x = (max([float(item) for item in x]))
        # x = float(x)
        if x>120.0 or x==0.0:
            x=-1.0
    except:
        x = -1.0
    return x

df['stories'] = df['stories'].apply(stories_float)

In [74]:
df['stories'].value_counts()

In [75]:
# df['stories'].value_counts().index.tolist()

In [76]:
# df['stories'].value_counts()[112.0:].sum()

#### Признак "homeFacts"

В первоначальном осмотре структуры датасет признаки 'homeFacts' и 'schools' имеют тексты, и содержат несколько отдельных сведений. Необходимо десериализовать содержимое этих признаков. Если имеются полезные данные, то необходимо из них создать новые признаки.

In [77]:
df['homeFacts'].value_counts()

In [78]:
# df['homeFacts'].value_counts().index.tolist()

Признак "homeFacts" содержит несколько отдельных данных, имеющих влияние на оценку недвижимости, дома, квартиры. Поэтому выделим эти данные. Для этого приведем содежимое данного признака к формату .json и формируем новые признаки.

In [79]:
df['homeFacts'] = df['homeFacts'].apply(lambda x: x.replace('None', '0'))
df['homeFacts'].value_counts()

In [80]:
# df['homeFacts'].value_counts().index.tolist()

In [81]:
# Для замены апострофы в словах
change_old = ["Attch'd", "Addtn'l",'"closet"', "Hennen\'s", "Penske's","Req'd"]
change_new = ['Attch`d','Addtn`l','`closet`', 'Hennen`s', 'Penske`s','Req`d']

for i in range(0,len(change_old)):
    df['homeFacts'] = df['homeFacts'].apply(lambda x: x.replace(change_old[i],change_new[i]))

In [82]:
df['homeFacts'] = df['homeFacts'].apply(lambda x: x.replace("'",'"'))

Проверка

In [83]:
df['homeFacts'][4]

In [84]:
ss = df['homeFacts'][4]
sa = json.loads(ss)
sa

In [85]:
# Для проверки правильности парсинга в json формат
for i in range(0,len(df)):
    ss = df['homeFacts'][i]
    try:
        sa = json.loads(ss)
    except:
        print(i)

In [86]:
def app(x,k):
    js = json.loads(x)
    return js['atAGlanceFacts'][k]['factValue']

In [87]:
df['Year built'] = df['homeFacts'].apply(lambda x: app(x,0))
df['Remodeled year'] = df['homeFacts'].apply(lambda x: app(x,1))
df['Heating'] = df['homeFacts'].apply(lambda x: app(x,2))
df['Cooling'] = df['homeFacts'].apply(lambda x: app(x,3))
df['Parking'] = df['homeFacts'].apply(lambda x: app(x,4))
df['lotsize'] = df['homeFacts'].apply(lambda x: app(x,5))
df['Price/sqft'] = df['homeFacts'].apply(lambda x: app(x,6))

#### Обработка новых выделенных признаков  из признака 'homeFacts'.

##### Признак 'Year built'

In [88]:
# df['Year built'] = df['homeFacts'].apply(lambda x: app(x,0))

In [89]:
df['Year built']

In [90]:
df['Year built'].value_counts()

In [91]:
df['Year built'] = df['Year built'].apply(lambda x: '0' 
                                    if ((x==0) or (x=='No Data') or (x=='') or len(x)>4) else x)
df['Year built'] = df['Year built'].astype(int)
df['Year built'].value_counts()

In [92]:
# df['Year built'].value_counts().index.tolist()

In [93]:
df[(df['Year built']>0) & (df['Year built']<1700)]

In [94]:
# Определим индексы "выбросов" в меньшую сторону.
ind_built_old= df[(df['Year built']>0) & (df['Year built']<1700)].index
ind_built_old

In [95]:
df['Year built'][ind_built_old]

In [96]:
df[df['Year built']>=2022]

In [97]:
# Определим индексы "выбросов" в большую сторону.
# Год строительства 2022 год тоже сомнительно!
ind_built_new = df[df['Year built']>=2022].index
ind_built_new

In [98]:
df['Year built'][ind_built_new]

In [99]:
df[df['Year built']==2025] # 163100, 285228

In [100]:
import matplotlib.pyplot as plt
import seaborn as sns

In [101]:
plt.rcParams['figure.figsize']=(15,10)

In [102]:
df['Year built'][df['Year built'] >1850].hist(bins=100)

##### Признак 'Remodeled year'

In [103]:
# df['Remodeled year'] = df['homeFacts'].apply(lambda x: app(x,1))

In [104]:
df['Remodeled year']

In [105]:
df['Remodeled year'].value_counts()

In [106]:
# df['Remodeled year'].value_counts().index.tolist()

In [107]:
df['Remodeled year'] = df['Remodeled year'].apply(lambda x: '0' if (x==0) or (x=='') else x)
df['Remodeled year'] = df['Remodeled year'].astype(int)
df['Remodeled year'].value_counts()

In [108]:
# Определим индексы "выбросов" в меньшую сторону.
ind_Remodeled_old= df[(df['Remodeled year']>0) & (df['Remodeled year']<1700)].index
ind_Remodeled_old

In [109]:
# Определим индексы "выбросов" в большую сторону.
ind_Remodeled_new= df[df['Remodeled year']>=2021].index
ind_Remodeled_new

In [110]:
df['Remodeled year'][ind_Remodeled_new]

In [111]:
df['Remodeled year'][df['Remodeled year']>1900].hist(bins=100)

Данные признаки 'Year built' и 'Remodeled year' лучше преобразовать в числовой вид, вычитая данные из максимального значения принака. Тогда нулевые значения признака будет иметь какой-то смысл, как возраст недвижимости.

****До определния разности исправим данные, или удалим ненужные строки.

In [112]:
df.loc[ind_built_old, 'Year built']

In [113]:
df.loc[ind_built_old]

In [114]:
df['Year built'][ind_built_new]

In [115]:
df[df['Year built']==2025]

In [116]:
df['Remodeled year'][ind_Remodeled_old]

In [117]:
df['Remodeled year'][ind_Remodeled_new]

Исправлению подлежат данные для индексов ind_built_old признака 'Year built' на ноль, так как остальные данные вполне приемлемы для использования в расчетах. Также можно обнулить для ind_Remodeled_old индексы  признака 'Remodeled year'

In [118]:
df.loc[ind_built_old, 'Year built'] = 0 

df.loc[ind_built_new, 'Year built'] = 2022

df.loc[ind_Remodeled_old, 'Remodeled year'] = 0

Заметили, что для индекса 278869  в признаке 'Year built' год постройки 2004, а ремонт в 'Remodeled year'- 1748 год, здесь  явная ошибка. Необходимо программно проверить для всех строк.

Чтобы полученные большие числа в результате вычитания "не потянули" результат для отсутствующих данных и нулей, поставим -1.0

In [119]:
max = df['Year built'].max()
df['Year built'] =  df['Year built'].apply(lambda x: float(max-x) if x!=0 else -1.0)
df['Year built']

In [120]:
max = df['Remodeled year'].max()
df['Remodeled year'] =  df['Remodeled year'].apply(lambda x: float(max-x) if x!=0 else -1.0)
df['Remodeled year'].value_counts()

In [121]:
df['Year built'][df['Year built']>0].hist(bins=50)

In [122]:
df['Remodeled year'][df['Remodeled year']>0].hist(bins=50)

Для индекса 278869  в признаке 'Year built' год постройки 2004, а ремонт в 'Remodeled year'- 1748 год, здесь  явная ошибка. Надо проверить.

##### Признак 'Parking'

In [123]:
df['Parking'] = df['homeFacts'].apply(lambda x: app(x,4))

In [124]:
df['Parking'].value_counts()

In [125]:
# Признак будет бинарный: 0 - нет паркинг, 1 - есть паркинг.
df['Parking']=df['Parking'].apply(lambda x: 0 
            if x==0 or x=='' or x=='No Data' or x=='No' else 1)        

df['Parking'].value_counts()

Была попытка систематизировать данные признака и сделать категориальным, но данные оказались слишким разнообразным.

Yes = ['Garage','Space', 'Parking', 'Carport', 'Car', 'Attached', 'Detached', 
       'Off Street', 'On Street', 'Onstr', 'Basement', 'Electric Door',
       'Driveway',  'Concr', 'Gravel', 'Assigned', 'Covered', 'Paved', 
       'Asphalt', 'Common', 'Alley', 'Open', 'Guest']
def pars_parking(x):
    if x==0 or x=='' or x=='No Data' or x=='No':
        x = '0'
    x = x.title()
    x = x.replace('Gravl', 'Gravel')
    x = x.replace('Drvwy', 'Driveway')
    x = x.replace('Aspht','Asphalt')
    x= x.replace('Spaces', 'Space')
    for i in range(0,len(Yes)):
        if -1!=x.find(Yes[i]):
            x = re.sub('[^0-9\.]', '', x).strip()
            # x = Yes[i]+ ' - '+x
            if len(x)==0:
                x = '1'
    return x

df['Parking'] = df['Parking'].apply(pars_parking)

top_Parking = df['Parking'].value_counts()[:30].index
df['Parking'] = df['Parking'].apply(lambda x: x if x in top_Parking else 'Yes')

##### Признак 'lotsize'

In [126]:
# df['lotsize'] = df['homeFacts'].apply(lambda x: app(x,5))

In [127]:
df['lotsize'].value_counts()

In [128]:
# df['lotsize'].value_counts().index.tolist()

Необходимо убрать единицы измерения, предварительно приведя их в единое измерение, и преобразовать признак в числовой.

In [129]:
# 1 acre = 43560 sqft 
sqft_acre = 43560

In [130]:
df['lotsize'][1055]

In [131]:
df['lotsize'] = df['lotsize'].apply(lambda x: '0' if (x==0) or (x=='') or 
                                    (x=='No Data') or (x=='—') or (x=='-- sqft lot') else x)

In [132]:
sf = ['sqft', 'sqft lot']
acr = ['Acres', 'acres lot', 'Acre', 'acre', 'acre lot']
def analyze(x):
    x = x.replace('Sq. Ft.', 'sqft')
    ed_izm = re.sub('[^A-Z a-z]', '', x).strip()
    value = re.sub('[^0-9,\.]', '', x).strip()
    if ed_izm in sf:
        x = x.replace(ed_izm, 'sqft')
    if ed_izm in acr:
        x = x.replace(ed_izm, 'acres')
    return x

In [133]:
df['lotsize'] = df['lotsize'].apply(analyze)

In [134]:
df['lotsize'].value_counts()

In [135]:
# df['lotsize'].value_counts().index.tolist()

In [136]:
def to_num(x):
    ed_izm = re.sub('[^A-Z a-z]', '', x).strip()
    value = re.sub('[^0-9\.]', '', x).strip()
    if ed_izm=='acres':
        value = round(float(value)*sqft_acre, 1)
    if ed_izm=='sqft'  or ed_izm=='':
        value = round(float(value), 1)
    return value

df['lotsize'] = df['lotsize'].apply(to_num) 

In [137]:
df['lotsize'].value_counts()

In [138]:
# df['lotsize'].value_counts().index.tolist()

In [139]:
df['lotsize'].describe()

In [140]:
df[df['lotsize']==df['lotsize'].max()]

In [141]:
# Число слишком большое, обнуляем.
df.loc[1148, 'lotsize'] = 0.0

##### Признак 'Price/sqft'

In [142]:
# df['Price/sqft']  = df['homeFacts'].apply(lambda x: app(x,6))

In [143]:
df['Price/sqft'].value_counts()

In [144]:
# df['Price/sqft'].value_counts().index.tolist()

In [145]:
df['Price/sqft']

In [146]:
df['Price/sqft'].isna().sum()

In [147]:
df['Price/sqft'] = df['Price/sqft'].fillna(0.0)

In [148]:
def is_num(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

def Price_sqft(x):
    if x==0 or x=='' or x=='No Data' or x=='No Info':
        x = '0'
        return float(x)
    x = re.sub('[^0-9\.]', '', x).strip()
    if is_num(x):
        return float(x)
    else:
        return 0.0

In [149]:
df['Price/sqft'] = df['Price/sqft'].apply(Price_sqft)

In [150]:
df['Price/sqft'].isna().sum()

In [151]:
df['Price/sqft'] = df['Price/sqft'].fillna(0.0)

Необходимо выявить выбросы и их удалить или исправить!
Признак 'Price/sqft' - числовой

In [152]:
df['Price/sqft'].describe()

In [153]:
plt.boxplot(df['Price/sqft'], vert=False)

In [154]:
df[['target','Price/sqft']][df['Price/sqft']>40000]

Предельная цена установилась подбором чисел: 20000, 30000, 40000, и т.д.
Цена кв.фута равна всей стоимости  недвижимости! Необходимо обнулять эти данные.

In [155]:
ind_Price_sq = df[['target','Price/sqft']][df['Price/sqft']>40000].index
ind_Price_sq

In [156]:
# =0.0
df.loc[ind_Price_sq, 'Price/sqft'] = 0.0

In [157]:
plt.boxplot(df['Price/sqft'], vert=False)

In [158]:
df['Price/sqft'].describe()

In [159]:
df['Price/sqft'].value_counts()

Признак 'Price/sqft' - числовой

#### Признаки 'Heating' и 'Cooling' анализируем совместно 

In [160]:
df['Heating'].value_counts()

In [161]:
# df['Heating'].value_counts().index.tolist()

In [162]:
df['Cooling'].value_counts()

In [163]:
# df['Cooling'].value_counts().index.tolist()

  Два признака, 'Heating' и 'Cooling', существенно влияют на цену недвижимости. Поэтому необходимо привести данные этих показателей в пригодную форму для использования в моделях прогноза цен на недвижимость.

In [164]:
# Функция для первоначального парсинга признаков 'Heating' и 'Cooling' 
patterns = "[!#$%&()'*+.:;<=>?@[\]^_`{|}~—\"]+"
def parsing (x):
    if (x==0) or (x=='0') or (x=='') or (x=='No Data'):
        x='No'
    x = x.title()
    x = re.sub(patterns, ' ', x)
    if len(x)>0 and x[0]==',':
        x = x[1:].strip()
    return x

In [165]:
df['Heating'] = df['Heating'].apply(parsing)
# df['Heating'].value_counts().index.tolist()

In [166]:
df['Heating'].value_counts()

In [167]:
df['Cooling'].value_counts()

In [168]:
# df['Cooling'].value_counts().index.tolist()

Беглый просмотр содержимого признаков 'Heating' и 'Cooling' показывает, что в признаке 'Cooling' приведены  тепловые показатели , а в признаке 'Heating' приведены показатели охлаждения.
Вначале исправим некоторые имеющиеся явно заметные неточности.

In [169]:
df[df['Heating']=='Central Air']

In [170]:
# 'Central Air'
#  Выделение индексов строк 
ind_H = df[df['Heating']=='Central Air'].index
df['Cooling'][ind_H].value_counts()

In [171]:
# Изменеие 'Has Cooling' признака 'Cooling' на 'Central Air' признака 'Heating'.
df.loc[ind_H,'Cooling'] = df['Heating'][ind_H]

In [172]:
df['Cooling'][ind_H].value_counts()

In [173]:
# Выделение индексов строк
ind_C =df[df['Cooling']=='Central Heating'].index
df['Heating'][ind_C].value_counts()

In [174]:
# Изменение  признака 'Heating' и  признака 'Cooling'.'Central Heating'
def change(x):
    if -1!=x.find('Gas'):
        x = 'Central Heating '+'Gas'
    if -1!=x.find('Electric '):
        x = 'Central Heating '+'Electric '
    return x

In [175]:
df.loc[ind_C, 'Heating'] = df['Heating'][ind_C].apply(change)
df['Heating'][ind_C].value_counts()

In [176]:
df.loc[ind_C,'Cooling']= 'Central'
df['Cooling'][ind_C].value_counts()

In [177]:
ind_CE =df[df['Cooling']=='Electric Heating'].index
df['Heating'][ind_CE].value_counts()

Содержимое признака 'Heating' выделенных строк больше похоже на данные признака 'Cooling'. Поэтому поменяем их между собой.

In [178]:
df.loc[ind_CE, 'Cooling'] = df['Heating'][ind_CE]

In [179]:
df.loc[ind_CE, 'Heating'] = 'Electric Heating'

In [180]:
df['Heating'][ind_CE].value_counts()

In [181]:
df['Cooling'][ind_CE].value_counts()

В дальнейшем содежимые этих признаков анализируются более подробно, и по мере возможности, очищаются, и данные приводятся к приемлемый вид для использования в моделях.

#### Признак 'Heating' отдельно

In [182]:
df['Heating'].value_counts()

In [183]:
# df['Heating'].value_counts().index.tolist()

In [184]:
fnd =['Air Conditioning','Air Conditionings','Air Conditioner','Air Conditioners','A/C','Ac']
   
def dop_Heat(x):
    if (-1!=x.find('Forced Air') or -1!=x.find('Heat Pump')) and -1!=x.find('Gas'):
        x='Forced Gas'
    if (-1!=x.find('Forced Air') or -1!=x.find('Heat Pump')) and -1!=x.find('Electric'):
        x='Forced Electric'
    if -1!=x.find('Heat Pump') or x=='Forcedair':
         x='Forced'
    if -1!=x.find('Forced Air') or x=='Forced Warm Air':
        x='Forced'
    for i in range(0, len(fnd)):
        x = x.replace(fnd[i], '')
    if -1!=x.find('Central Air'):
        x='Central'
    if -1!=x.find('Central') and -1!=x.find('Electric'):
        x='Central Electric'
    if -1!=x.find('Central') and -1!=x.find('Gas'):
        x='Central Gas'
    if x== 'Electric':
        x='Central Electric'
    return x


In [185]:
df['Heating'] = df['Heating'].apply(dop_Heat)

In [186]:
gas = ['Gas', 'Forced Gas', 'Central Gas', 'Electric, Gas']
central = ['Central Electric', 'Central Gas', 'Central Furnace', 'Central Heating']
electric = ['Central Electric', 'Forced Electric', 'Electric, Gas'] 
name_change = ['Wall', 'Refrigeration', 'Baseboard', 'Hot Water', 'Stove', 'Radiator']
def in_gce(x):
    x = x.strip()
    x = x.replace('Units', 'Unit')
    x = x.replace('Unit S', 'Unit')
    if ((-1!=x.find('Gas') or -1!=x.find('Propane')) and (x not in gas)):
        x='Gas'
    if (-1!=x.find('Central') and (x not in central)):
        x='Central'
    if (-1!=x.find('Electric') and (x not in electric)):
        x='Electric'
    for i in range(0, len(name_change)):
        if -1!=x.find(name_change[i]):
            x = name_change[i]
    if -1!=x.find('Heat') and -1==x.find('Heating'):
        x='Heating'
    if -1!=x.find('Heating') and (x not in ['Central Heating', 'Heating System']):
        x='Heating'
    if -1!=x.find('Window Unit') or x=='Window':
        x = 'Window Unit'
    if  -1!=x.find('Hot Air'):
        x='Forced'
    return x

In [187]:
df['Heating'] = df['Heating'].apply(in_gce)

In [188]:
df['Heating'].value_counts()

In [189]:
df['Heating'].value_counts().index.tolist()

In [190]:
df['Heating'].value_counts()['Stove':].sum()

Данные признака 'Heating' оказались слишком разнообразными. Требовалось детально изучать данные по строкам. Данные по возможности приведены к удобному виду для категоризации данных.  
Все остальные значения после  признака 'Stove' отнесем к 'Other'

In [191]:
ind_Heat = df['Heating'].value_counts()[:'Stove'].index
ind_Heat

In [192]:
df['Heating'] = df['Heating'].apply(lambda x: x 
                                            if x in ind_Heat else 'other')

In [193]:
df['Heating'].value_counts()

#### Признак 'Cooling' отдельно

In [194]:
df['Cooling'].value_counts()

In [195]:
# df['Cooling'].value_counts().index.tolist()

In [196]:
df['Cooling'] = df['Cooling'].apply(parsing)

In [197]:
df['Cooling'].value_counts().index.tolist()

In [198]:
fnd =['Air Conditioning','Air Conditionings','Air Conditioner','Air Conditioners','A/C','Ac']
name_change = ['Central Air','Hot Air','Window Unit','Refrigeration','Air Purification System',
               'Common Heating/Cooling', 'Zoned','Baseboard','Wall','Other','No']
def dop_Cool(x):
    x = x.strip()
    x = x.replace('Fan S', 'Fan')
    x = x.replace('Unit S', 'Unit')
    for i in range(0, len(fnd)):
        x = x.replace(fnd[i], 'Air Conditioning')
    if -1!=x.find('Air Conditioning'):
        x='Air Conditioning'
    if -1!=x.find('Central') and -1==x.find('Air'):
        x='Central Air'
    for i in range(0, len(name_change)):
        if -1!=x.find(name_change[i]):
            x = name_change[i]
    if -1!=x.find('Central') and -1!=x.find('Electric'):
        x='Central Electric'
    if -1!=x.find('Central') and -1!=x.find('Gas'):
        x='Central Gas'
    if x== 'Electric':
        x='Central Electric'
    if -1!=x.find('Heat Pump') or -1!=x.find('Forced Air'):
        x='Forced Air'
    if -1!=x.find('Window') and -1==x.find('Unit'):
        x = 'Window Unit'
    if -1!=x.find('Ceiling'):   
        x = 'Ceiling Fan'
    if -1!=x.find('Air Purification System'):    
        x = 'Air Purification System'
    if -1!=x.find('Evaporative'):    
        x = 'Evaporative Cooling'
    return x

In [199]:
df['Cooling'] = df['Cooling'].apply(dop_Cool)

In [200]:
df['Cooling'].value_counts()

In [201]:
# df['Cooling'].value_counts().index.tolist()

Все остальные значения после  с признака 'Zoned' отнесем к 'Other'
Непонятно, почему-то в признаке 'Cooling' данные чаще всего описывают о теплоснабжении!?
Предполагается,что некоторые установки для тепла также используются для охлаждения.
Данные приведены к удобному виду для категоризации данных, по возможности.

In [202]:
ind_Cool = df['Cooling'].value_counts()['Zoned':].index
ind_Cool

In [203]:
df['Cooling'] = df['Cooling'].apply(lambda x: x if x in ind_Cool else 'Other')

In [204]:
df['Cooling'].value_counts()['Zoned':].sum()

In [205]:
df['Cooling'].value_counts()['Other']

In [206]:
# df['Heating_Cooling'] = df['Heating']+', '+df['Cooling']
# df['Heating_Cooling'] = df['Heating_Cooling'].apply(parsing)

In [207]:
# df['Heating_Cooling'].value_counts()

In [208]:
# df['Heating_Cooling'].value_counts().index.tolist()

#### Признак "schools"

Признак "schools" как и признак "homeFacts" также  содержит  отдельные данные, имеющие влияние на оценку недвижимости (дома, квартиры). Поэтому выделим эти данные. Для этого также приведем содежимое данного признака к формату .json и сформируем новые признаки.

In [209]:
# df['schools']=data['schools']

In [210]:
df['schools'] = df['schools'].apply(lambda x: x.replace('None', '0'))

In [211]:
df['schools'].value_counts()

In [212]:
# df['schools'].value_counts().index.tolist()

Предварительный обзор данных показывает необходимость подробного парсинга признака. В ручную подберем символов, подлежащие изменению.

In [213]:
# Просмотр строк с апострофом среди текста, например, "Lil\' Wonders Development Center Private School" 
ss = df['schools'].loc[1485]
ss

In [214]:
old = ["O\'", " O`,",  "'s", "\'Duke\'", "B\'Nai", "B\'inai",  "L\'Ouver", "L\'ouver",  "L\'Banos", "L\'Dovid", 
              "Lil\'", "D\'",  "d\'Evelyn", "Q\'", "Loggers\'", "Teach N\'",  "K\'Tana", "R&D`",  "Citizens\'",  "Kids\'", 
              "Girls\' ", "d\'Chassidei", "d\'Gur", "d\'Rav", "Pe\'Er", "Be\'Er", "Kids \'R\' Kids", "\'joe\' ", 
              "Meadow\'S", "Sheriff\'S", "Friends\' ", "\'Mother\'", "Int\'L", "\'buddy\'"]
new = ['O`',  " O\', ", '`s', '`Duke`', 'B`Nai', 'B`inai',  'L`Ouver', 'L`Ouver', 'L`Banos', 'L`Dovid', 
              'Lil`', 'D`', 'd`Evelyn', 'Q`', 'Loggers`', 'Teach N`', 'K`Tana', "R&D\'",   'Citizens`', 'Kids`', 
              'Girls` ',  'd`Chassidei','d`Gur', 'd`Rav', 'Pe`Er', 'Be`Er', 'Kids `R` Kids', '`joe` ', 
              'Meadow`S', 'Sheriff`S', 'Friends` ', '`Mother`', 'Int`L', '`buddy`']

In [215]:
for i in range(0,len(old)):
    df['schools'] = df['schools'].apply(lambda x: x.replace(old[i], new[i]))

df['schools'] = df['schools'].apply(lambda x: x.replace("'",'"'))

In [216]:
# Проверка после преобразования отдельных строк с неправильными данными
ss = df['schools'].loc[1485]
sa = json.loads(ss)
sa

In [217]:
ss = df['schools'].loc[1492][1:len(df['schools'][1492])-1]
sa = json.loads(ss)
sa

In [218]:
# Для проверки правильности парсинга признака 'schools' в json формат
for i in range(0,len(df)):
    ss = df['schools'].loc[i][1:len(df['schools'][i])-1]
    try:
        sa = json.loads(ss)
    except:
        print(i)

Создание новых признаков из признака 'schools'

In [219]:
def app_schools(x,k):
    js = json.loads(x[1:len(x)-1])
    if k==0:
        return js['rating']
    if k==1:
        return js['data']['Distance']
    if k==2:
        return js['data']['Grades']
    if k==3:
        return js['name']

In [220]:
df['schools_rating'] = df['schools'].apply(lambda x: app_schools(x,0))
df['schools_Distance'] = df['schools'].apply(lambda x: app_schools(x,1))
df['schools_Grades'] = df['schools'].apply(lambda x: app_schools(x,2))
df['schools_name'] = df['schools'].apply(lambda x: app_schools(x,3))

#### Новые признаки  анализируются по отдельности
'schools_rating', 'schools_Distance', 'schools_Grades', 'schools_name'

Сперва посчитаем количества школ и создаем отдельный признак "school_count"

In [221]:
df['school_count'] = df['schools_rating'].apply(lambda x: len(x))

In [222]:
df['school_count'].value_counts()

In [223]:
df['school_count'].value_counts().index.tolist()

##### Признак  'schools_rating'

In [224]:
# Вычисляется средний рейтинг школ
df['schools_rating']

In [225]:
# df['schools_rating'].value_counts().index.tolist()

In [226]:
def rait_ch(x):
    count = len(x)
    c = 0
    s = 0
    for i in range(0, len(x)):
        x[i]=x[i].replace('/10', '')
        if x[i].isdigit():
            s+=int(x[i])
            c+=1
    if c!=0:
        s = round(s/c, 2)
    else:
        s = 0
    return s

In [227]:
df['schools_rating'] = df['schools_rating'].apply(rait_ch)

In [228]:
# df['schools_rating'].value_counts().index.tolist()

In [229]:
df['schools_rating'].value_counts()

##### Признак  'schools_Distance'

In [230]:
# Вычисляется средняя дистанция до школ
df['schools_Distance']

In [231]:
def dist_ch(x):
    count = len(x)
    c = 0
    s = 0
    for i in range(0, len(x)):
        x[i]=x[i].replace('mi', '')
        if float(x[i]):
            s+=float(x[i])
            c+=1
    if c!=0:
        s = round(s/c, 2)
    else:
        s = 0
    return s

In [232]:
df['schools_Distance'] = df['schools_Distance'].apply(dist_ch)

In [233]:
# df['schools_Distance'].value_counts().index.tolist()

In [234]:
df['schools_Distance']

##### Признак  'schools_Grades'

In [235]:
df['schools_Grades']

In [236]:
df['schools_Grades'].value_counts()

In [237]:
# Для приведения к единой структуре данных 
def change_to(x):
    if len(x)!=0:
        for i in range(0, len(x)):
            try:
                x[i] = str(x[i].replace(' to ','-'))
            except:
                if x[i]==0: x[i]='0'
                else: print(x)
    else:
        x = []
    return x

In [238]:
df['schools_Grades'] = df['schools_Grades'].apply(change_to)

In [239]:
df['schools_Grades'].value_counts()

In [240]:
# df['schools_Grades'].value_counts().index.tolist()

Цель преобразования:  данные признака  привести к форму 

"Самое младшее образовательное учреждение" - "Самое старшое образовательное учреждение"

In [241]:
def corr(x):
    s = set()
    for i in range(0, len(x)):
        if [x[i]]==x[i].split(','):
            ch = re.sub(r'[0-9a-zA-Z]', '', x[i])
            if ch!='':
                s.add(x[i].split(ch)[0])
                if len(x[i].split(ch))>1:
                    s.add(x[i].split(ch)[1])
            else:
                s.add(x[i])
        else:
            a = x[i].split(',')
            ch = re.sub(r'[0-9a-zA-Z]', '', a[0])
            if ch!='':
                s.add(a[0].split(ch)[0])
                if len(a[0].split(ch))>1:
                    s.add(a[0].split(ch)[1])
            else:
                s.add(a[0])
            ch = re.sub(r'[0-9a-zA-Z]', '', a[1])
            if ch!='':
                try:
                    s.add(a[1].split(ch)[0])
                except:
                    print(ch,'  ', a)
                if len(a[1].split(ch))>1:
                    s.add(a[1].split(ch)[1])
            else:
                s.add(a[1])
    return s

df['schools_Grades'] = df['schools_Grades'].apply(corr)

In [242]:
df['schools_Grades'].value_counts()

In [243]:
# df['schools_Grades'].value_counts().index.tolist()

In [244]:
left = ['Preschool', 'PK', 'K']
def diapason_school(x):
    x = list(x)
    stroka=''
    if len(x)==0:
        stroka = '0'
    else:
        x.sort()
    # Предполагаемое самое младшее образовательное учреждение
        a = x[-1]
        max=0
        min=12
    # Выделение самого старшего класса
        for i in range(0, len(x)):
            try:
                if max<int(x[i]):
                    max = int(x[i])
                if min>int(x[i]):
                    min=int(x[i])
            except:
                continue
        if a in left:
            stroka = a+' - '+str(max)
        else:
            stroka = str(min)+' - '+str(max)
    return stroka

df['schools_Grades'] = df['schools_Grades'].apply(diapason_school)

In [245]:
# df['schools_Grades'].value_counts().index.tolist()

In [246]:
df['schools_Grades'].value_counts()

Признак категориальный

##### Признак  'schools_name'

In [247]:
# Данный признак подлежит удалению.
df['schools_name']

In [248]:
df.drop(['schools_name'], axis=1, inplace=True)

#### Признак "PrivatePool" & "private pool"

In [249]:
df.columns

In [250]:
df['private pool'].value_counts()

In [251]:
df['PrivatePool'].value_counts()

In [252]:
df['private pool'] = df['private pool'].apply(lambda x: 1 if x=='Yes' else 0)
df['PrivatePool'] = df['PrivatePool'].apply(lambda x: 1 if (x=='Yes' or x=='yes') else 0)

In [253]:
df['PrivatePool'].value_counts()

In [254]:
df['private pool'].value_counts()

In [255]:
# Объединим два признака:
df['PrivatePool'] = df['private pool'] | df['PrivatePool']
df.drop(['private pool'], axis=1, inplace=True)

In [256]:
df['PrivatePool'].value_counts()

#### Признак "fireplace"

Содержимое признака "fireplace" также небрежно заполнено и данные очень разнообразны, поэтому требуют сложного и кропотливого парсинга содежимого признака.

In [257]:
df['fireplace'] = df['fireplace'].fillna('0') # Заполнение 0 пустых признаков, отсутствие камин 
df['fireplace'] = df['fireplace'].apply(lambda x: x.lower()) #  Перевод всех символов к нижнему регистру

In [258]:
no = ['no fireplace', 'no', 'n/k'] # Признаки  отсутствия камин
for i in range(0, len(no)):
    df['fireplace'] = df['fireplace'].apply(lambda x: '0' if x==no[i] else x)

In [259]:
df['fireplace'] = df['fireplace'].apply(lambda x: '1' if x=='yes' else x) # Есть камин, считаем,  что единственный

In [260]:
# Для подсчета каминов в доме используем слово "room". 
# Для этого в названиях комнат расшифруем сокращения и пропущенное слово "room", заодно расшифруем слово 'fp'
old= ['fp', 'bedroom',  'familyrm', 'family rm', 'livingrm', 'living rm', 'guest house',
      'kitchen',  'basement',  'in-law quarters'] 

new = ['fireplace',   'bed room', 'family room', 'family room', 'living room', 'living room', 'guest house room',
'kitchen room', 'basement room',  'in-law quarters room']

for i in range(0, len(old)):
    df['fireplace'] = df['fireplace'].apply(lambda x: x.replace(old[i], new[i]))

In [261]:
# Здесь слово из yes_substr заменяется на слово из in_str , если отсуствует некотроые сопутствующее слово из non_substr.
yes_substr = ['master', 'dining', 'den']
non_substr = ['bed room', 'kitchen/dining room', 'den/family']
in_str = ['master bed room', 'dining room', 'den room'] 
for i in range(0,len(yes_substr)):
    df['fireplace'] = df['fireplace'].apply(
        lambda x: x.replace(yes_substr[i], in_str[i]) if ((-1!=x.find(yes_substr[i])) and (-1==x.find(non_substr[i]))) else x)

In [262]:
#  Переводим словесное описание чисел в цифровой вид и заменим всю строку
n_old = ['one', 'two', 'three', 'four'] 
n_new = ['1','2', '3', '4']
for i in range(0, len(n_old)):
    df['fireplace'] = df['fireplace'].apply(lambda x: n_new[i] if (-1!=x.find(n_old[i])) else x)

In [263]:
sub_str = ['fireplace','gas log']
for i in range(0,len(sub_str)):
    df['fireplace'] = df['fireplace'].apply(lambda x: '1' if -1!=x.find(sub_str[i]) else x)

In [264]:
# Подсчет количества комнат, где имеется камин
df['fireplace'] = df['fireplace'].apply(lambda x: str(x.count('room'))+' '+x if 0<x.count('room') else x)

df['fireplace'] = df['fireplace'].apply(
        lambda x: '2' if (-1!=x.find('2') and  -1==x.find('2+') and -1==x.find('12')) else x)

df['fireplace'] = df['fireplace'].apply(
        lambda x: '1' if (-1!=x.find('1') and -1==x.find('10') and -1==x.find('11') and -1==x.find('12')) else x)

In [265]:
# Замена всей строки на элемент из n_new  при отсутсвии сопутствущего слово из n_not
n_new = ['3', '4', 'ceiling fan']
n_not = ['3+', '4+', 'fireplace']
for i in range(0,len(n_new)):
    df['fireplace'] = df['fireplace'].apply(
        lambda x: n_new[i] if (-1!=x.find(n_new[i]) and -1==x.find(n_not[i])) else x)

In [266]:
n = ['2+', '3+', '4+', '5', '6', '7', '8', '9' , '10', '11', '12']
for i in range(0, len(n)):
    df['fireplace'] = df['fireplace'].apply(lambda x: n[i] if (-1!=x.find(n[i])) else x)

df['fireplace'] = df['fireplace'].apply(lambda x: '0' if -1!=x.find('utility connection') else x)

In [267]:
df['fireplace'].value_counts()

Создаем список вручную, скопировав распечатки Type.index.tolist(), и вручную выберем индексы-наименования, показывающие  отсутствование камина, и создаем список fp_no.
Затем вычитаем из общего списка содержимого fp_no, и из оставшегося списка выберем также вручную все индексы-наименования  с числами и создаем список fp_yes.
Предполагаем, что оставшийся список показывает наличия хотя бы одного камина.
Из списков fp_no и создаем файлы: "fireplace_no" и "fireplace_yes".

Пришлось обойтись без лемминга и токенайзера.

In [268]:
Type = df['fireplace'].value_counts()
s = Type.index.tolist()
# s

In [269]:
# Из списка s вручную выделим строки датасет,  где предполагается 
#  отсутствие камина ('fireplace'), и переименуем списка как  fp_no
fp_no =['not applicable',
 'ceiling fan',
 'smthr',
 'smapl',
 'number',
 'extra closets, walk-in closets',
 'extra closets, smthr, storage, walk-in closets',
 'redecorated',
 'other-see remarks',
 'air filter, storage, walk-in closets',
 'extra closets, smthr, walk-in closets',
 'smthr, storage, walk-in closets',
 'air filter',
 'see through',
 'elevator',
 'extra closets, smthr',
 'storage, wet bar',
 'extra closets, redecorated, storage',
 'fire sprinkler system, smcam, smlt, smthr, storage, walk-in closets',
 'fire sprinkler system, redecorated',
 'central vacuum, elevator, extra closets, walk-in closets, wet bar',
 'water filter',
 'air filter, extra closets, smthr, walk-in closets',
 'heating system: yes(s)',
 'water filter, wet bar',
 'central vacuum, elevator, walk-in closets, wet bar',
 'smcam',
 'extra closets, fire sprinkler system',
 'central vacuum, smthr',
 'extra closets, storage, walk-in closets, water filter, wet bar',
 'redecorated, storage',
 'air filter, dehumidifier, extra closets, redecorated, storage, water filter',
 'fire sprinkler system, smthr',
 'extra closets, storage, walk-in closets, wet bar',
 'extra closets, fire sprinkler system, redecorated',
 'air filter, walk-in closets, water filter, wet bar',
 'central vacuum, elevator, extra closets, hot tub, smthr, wet bar',
 'extra closets, smcam, smthr',
 'elevator, fire sprinkler system, storage, walk-in closets',
 'walk-in closets, water filter, wet bar',
 'hot tub, redecorated',
 'redecorated, walk-in closets',
 'extra closets, smapl',
 'presently inoperative, decorative only',
 'other - see remarks',
 'air filter, central vacuum, intercom, walk-in closets',
 'firepit',
 'fire pit,gas fuel,location',
 'fire pit, location',
 'firepit, wood',
 'central vacuum',
 'central vacuum, elevator, extra closets, storage, walk-in closets, wet bar',
 'storage, walk-in closets, water filter, wet bar',
 'fire pit,see through,location',
 'air filter, central vacuum, elevator, extra closets, redecorated, wet bar',
 'air filter, elevator, extra closets, walk-in closets, wet bar',
 'elevator, walk-in closets',
 'central vacuum, extra closets, smthr, storage, walk-in closets',
 'elevator, extra closets, fire sprinkler system, storage, walk-in closets',
 'air filter, extra closets, storage, walk-in closets',
 'dehumidifier, fire sprinkler system, storage, walk-in closets, water filter']

In [270]:
f = open("fireplace_no", "w")
for  line in fp_no:
    f.write(line)
    f.write('\n')
f.close()

In [271]:
s1 = list(set(s) - set(fp_no))
# s1

In [272]:
fp_yes = ['natural gas, propane logs convey',
 'elevator, storage, walk-in closets',
 'wood burning, gas',
 'wood burning w/gas starter',
 'bath features',
 'redecorated, storage, walk-in closets',
 'decorative,fire pit,location',
 'recirculator, wood',
 'gas fuel,wood fuel,location',
 'factory built, gas starter',
 'wood, location',
 'wood burning',
 'insert unit, blower',
 'fire sprinkler system',
 'extra closets, fire sprinkler system, storage, walk-in closets',
 'gas vented',
 'wood stove',
 'attached doors, ventless',
 'factory built, metal, wood stove',
 'insert, wood',
 'extra closets, storage, walk-in closets',
 'wood burning, woodstove',
 'fire pit,free standing,electric,location',
 'outside, wood burning',
 'woodburning',
 'wood fuel,location',
 'gas, wood burning',
 'wood burning, includes accessories',
 'brick, wood burning, wood stove',
 'factory built, metal',
 'central vacuum, extra closets, redecorated, smthr, walk-in closets',
 'electric logs',
 'free standing,location',
 'ventless, propane',
 'wood',
 'gas starter',
 'prefab',
 'free-standing, insert, wood',
 'flue',
 'gas, location',
 'gas, propane logs convey',
 'electric, wood burning',
 'gas',
 'gas, ventless, propane',
 'gas, wall mounted',
 'woodburning, gas starter, living, freestanding',
 'gas starter, portable/ventless',
 'metal',
 'insert, recirculator, wood',
 'decorative, electric, wood burning',
 'decorative',
 'free standing',
 'wood burning, heatilator type system, includes accessories',
 'fire pit,location',
 'ventless',
 'free standing, wood burning',
 'factory built, wood stove',
 'free standing, propane',
 'bath',
 'wood burning, electric, gas starter',
 'in living area, wood burning',
 'living',
 'gas, attached doors, flue',
 'wood burning, gas starters, ventless',
 'wall mounted, electric',
 'decorative,gas fuel',
 'gas burning, woodburning',
 'gas unvented, propane',
 'gas starters',
 'gas, gas starters',
 'coal',
 'electric logs, outside',
 'free standing wood stove',
 'insert',
 'decorative, wood burning',
 'decorative, electric, non wood burning',
 'other (see remarks)',
 'intercom, storage',
 'propane stove, freestanding',
 'wood fuel',
 'marble, stove',
 'decorative, gas',
 'wood burning frplc',
 'gas fuel,location',
 'gas starter, masonry',
 'extra closets, redecorated, storage, walk-in closets',
 'wet bar',
 'see through,location',
 'factory built',
 'gas, gas starters, free standing',
 'propane, wood burning',
 'vented gas',
 'decorative only',
 'gas, propane',
 'gas, ventless',
 'special features',
 'extra closets, fire sprinkler system, redecorated, storage',
 'decorative,location',
 'gas burning',
 'extra closets, smcam, smlt, smthr, storage, wet bar',
 'unit features',
 'extras',
 'extra closets, storage',
 'woodstove',
 'wood stove insert',
 'smcam, smlt, smthr, storage, walk-in closets',
 'brick',
 'decorative, electric, free standing',
 'walk-in closets, wet bar',
 'decorative,gas fuel,location',
 'outdoors',
 'gas unvented',
 'brick, flue, wood burning stove',
 'brick, wood burning',
 'in living area',
 'wood burning, electric logs',
 'log wood-burning',
 'decorative, other (see remarks)',
 'free-standing, wood',
 'location',
 'stove, wood burning',
 'see agent',
 'extra closets',
 'gas vented, natural gas',
 'pellet',
 'electric',
 'masonry',
 'pre-fab',
 'decorative, electric',
 'woodburning stove',
 'smthr, walk-in closets',
 'gas fuel,wood fuel',
 'gas, gas starters, attached screens',
 'fire sprinkler system, smlt, smthr',
 'pellet, wood',
 'walk-in closets',
 'wood burning, gas, flue',
 'log woodburning',
 'gas fuel',
 'gas fuel,gas starter,location',
 'gas, attached screens, ventless',
 'wood-burning stove',
 'storage',
 'gas, wood stove, ventless',
 'heatilator type system',
 'free standing frplc',
 'wood burning, gas starter',
 'pellet stove',
 'masonry, wood stove',
 'electric, non wood burning',
 'electric, free standing',
 'fire pit',
 'gas, gas starters, ventless',
 'brick, free standing',
 'gas, gas starters, ventless, propane',
 'in living area, wood burning, free standing',
 'wood burning stove',
 'direct vent',
 'gas, attached screens',
 'storage, walk-in closets',
 'electric logs, free standing',
 'gas fuel,gas starter',
 'factory built, masonry',
 'electric,location',
 'study',
 'natural gas',
 'gas, non wood burning',
 'electric,gas fuel,location']

In [273]:
f = open("fireplace_yes", "w")
for  line in fp_yes:
    f.write(line)
    f.write('\n')
f.close()

In [274]:
#  Переменное содержит строки, по смыслу указывающие на отсутствие камина. 
f = open("fireplace_no", "r")
fp_0 = f.readlines()
f.close()

In [275]:
## Переменное содержит строки, по смыслу указывающие на наличие  хотя бы одного камина.
f = open("fireplace_yes", "r")
fp_1 = f.readlines()
f.close()

In [276]:
for i in range(0, len(fp_0)):
    df['fireplace'] = df['fireplace'].apply(lambda x: '0' if x== fp_0[i].strip() else x)
for i in range(0, len(fp_1)):
    df['fireplace'] = df['fireplace'].apply(lambda x: '1' if x== fp_1[i].strip() else x)

In [277]:
# Признак сделаем числовым
df['fireplace'] = df['fireplace'].apply(lambda x: int(re.sub('\+','',x)))

In [278]:
df['fireplace'].value_counts()

Признак оказался самым сложным. 

#### Признак "state"

In [279]:
df['state'].value_counts()

In [280]:
df['state'] = df['state'].apply(lambda x: x.upper())
df['state'].unique()

In [281]:
df['state'].value_counts(ascending = True).plot(kind = 'barh')

#### Признак "city"

In [282]:
df['city'].value_counts()

In [283]:
# df[df['city'].isna()].sum()

In [284]:
df['city'] = df['city'].fillna('other')
top_city = df['city'].value_counts()[:200].index

df['city'] = df['city'].apply(lambda x: x if x in top_city else 'other')

In [285]:
df['city'].value_counts()

Признак "city" - категориальный. Если категориальных признаков будут слишком много, то придется уменьшить число категории признака "city", Если необходимо, то категории можно и увеличить.

#### Признак "sqft"

In [286]:
df['sqft'].value_counts()

In [287]:
# df['sqft'].value_counts().index.tolist()

In [288]:
df.sqft.isna().sum()

In [289]:
df.sqft = df.sqft.fillna('0')

In [290]:
def make_features_sqft(x):
    x = x.replace('--', '0')
    x = re.sub('[^0-9]', '', x)
    return x

df.sqft = df.sqft.apply(make_features_sqft)

In [291]:
# df.sqft.value_counts().index.tolist()

In [292]:
df['sqft'] = df['sqft'].apply(lambda x: '0' if x=='' else x)
df['sqft'] = df['sqft'].astype(np.float)

# df.sqft.value_counts().index.tolist()

In [293]:
df.sqft.value_counts()

Всем пропущенным значениям поставлено число 0. Этот признак не очень согласуется с признаками "beds" и "baths".

Признак  "sqft" - числовой

In [294]:
df['sqft'] = df['sqft'].apply(lambda x: -1.0 if x=='' or x=='0' else x)
df['sqft'].astype(np.float)

Всем пропущенным значениям проставлено число 0. Этот признак не очень согласуется с признаками "beds" и "baths".

Признак  "sqft" - числовой

#### Признак "baths"

In [295]:
df['baths'].value_counts()

In [296]:
# df['baths'].value_counts().index.tolist()

В результате детального анализа признака данные приведены к более или менее единообразию. 
Значение поля "Bathrooms: 241"   в признаке "baths" по всей вероятности "Bathrooms: 4", так как является домом с 4-мя спальнями для одной семьи. Поэтому ставим 4 вместо 241. Bathrooms: SemiMod ==> "Bathrooms: 6", и т.д. Все изменения в функции  make_baths(x).

In [297]:
def is_num(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

def make_baths(x):
    x = re.sub('ba','Baths', x)
    x = re.sub('1 1/2', '1.5', x)
    x = x.replace('.0', '')
    x = x.replace('Bathsths','Baths')
    x = x.replace('7,500+','7,500')
    if x == 'Bathrooms: 241': 
        x='4 Baths'
    if x == 'Bathrooms: SemiMod': 
        x='6 Baths'
    if x == 'Sq. Ft. ':
        x = '0'
    x = x.replace('Bathrooms: ','')
    x = x.replace('Bathrooms: ','')
    x = x.replace('+','+ Baths')
    if x=='1 / 1-0 / 1-0 / 1-0' or x=='2-1 / 2-1 / 1-1 / 1-1':  
        x = '2 Baths'
    if x=='1 / 1 / 1 / 1' or x=='1-0 / 1-0 / 1' or x=='3-1 / 2-2':  
        x = '3'
    if x=='116 / 116 / 116':
        x='116 Baths'
    if x=='0 / 0' or x== '0.00' or x=='0.0' or x=='00' or x=='0 /':
        x='0'
    if (-1.0!=x.find('--') or -1.0!=x.find('~') or -1.0!=x.find('—')):
        x = '0'
    if is_num(x) and float(x)<120:
        x = x + ' Baths'
    return x

In [298]:
df.baths = df.baths.fillna('0')
df.baths = df.baths.apply(make_baths)

In [299]:
df.baths.value_counts()

In [300]:
# df.baths.value_counts().index.tolist()

In [301]:
df.baths.value_counts()['44 Baths':].sum()

In [302]:
ind_Baths_top = df.baths.value_counts()[:50]

Признак 'baths' - категориальный, так как данные заданы и в количестве и в виде  площади.

#### Признак "beds"

In [303]:
df['beds'].value_counts()

In [304]:
# df['beds'].value_counts().index.tolist()

In [305]:
df.beds = df.beds.fillna('0')

In [306]:
df['beds'].value_counts()

In [307]:
# df['beds'].value_counts().index.tolist()

In [308]:
# Выделим из признака 'beds' идексов строк датасет (df), где заначением являются 'Bath' или 'Baths'
# Для этого создаем временный служебный признак
df['beds_Baths'] = df['beds'].apply(lambda x: x if -1!=x.find('Bath') else ' ')

In [309]:
# Выделим  и сгруппируем для этих индексов данные признака 'baths'
ind_Baths =  df[df['beds_Baths']!=' '].index

In [310]:
# Удалим служебный признак
df.drop('beds_Baths', axis=1, inplace=True)

In [311]:
df['beds'][ind_Baths].value_counts()

Заметим, что  3356  строк имеют значения 0, хотя в признаке есть "Baths". Заменим их на "1 Bath"

In [312]:
df.loc[ind_Baths, 'baths'] = df['baths'][ind_Baths].apply(lambda x: '1 Bath' 
                                                          if x== '0 Baths' else x)

In [313]:
df['baths'][ind_Baths].value_counts()

In [314]:
df['beds'][ind_Baths].value_counts().index.tolist()

In [315]:
df['beds'][ind_Baths].value_counts()['2 Baths, 3 or More Bedrooms']

In [316]:
def make_beds(x):
    x = x.replace('bd','Beds')
    x = x.replace('1-2','2')
    x = x.replace('.0', '')
    x = x.replace('--', '0')
    if x=='Baths':
        x='Bath'
    if len(x)>0 and len(x)<=3 and x!='0':
        x = x + ' Beds'
    if( x==' ') or (x=='0 sqft') or (x=='0'):
        x='0 Beds'
        
    if x==' Beds' or x=='  Beds':
        x='1 Beds'
    if -1!=x.find('3 or More Bedrooms'):
        x = '3+ Beds' 
    if -1!=x.find('2 Bedrooms'):
        x = '2 Beds' 
    return x

In [317]:
df.beds = df.beds.apply(make_beds)

In [318]:
df['beds'].value_counts()

In [319]:
# df['beds'].value_counts().index.tolist()

In [320]:
# Имеются длинные данные, которые не имеют отношения к данному признаку. Их два,поставим 0.
df.beds = df.beds.apply(lambda x: '0 Beds' if len(x)>25 else x)

Здесь небходим полный анализ данных, так как в признаке о спальнях присутствуют несвойственные данные для спальных комнат в акрах, измеримые для оценки земельных участков. Размеры в кв. футах допустимы, надо найти способ совместить с количеством комнат.
* 1 acre = 4046,856 кв.м.
* 1 acre = 43560 sqft 
* 1 sqft = 0,092903 кв.м.
* acre_sqft = 43560

In [321]:
def is_num(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

acre_sqft = 43560

In [322]:
def acre_to_sqft(x):
    ed_izm = re.sub('[^A-Z a-z]', '', x).strip()
    value = re.sub('[^0-9\.]', '', x).strip()
    if ed_izm=='acres' or ed_izm=='acre':
        if is_num(value):
            value = round(float(value)*acre_sqft, 1)
            return str(value)+' '+ 'sqft'
        else:
            return value
    if ed_izm=='sqft'  or ed_izm=='':
        if is_num(value):
            value = round(float(value), 1)
            return str(value)+' '+ 'sqft'
        else:
            return value
    if ed_izm =='Beds' or ed_izm=='Bath':
        return x

In [323]:
df['beds'] = df['beds'].apply(acre_to_sqft)

In [324]:
df['beds'].value_counts()

In [325]:
# df['beds'].value_counts().index.tolist()

Признак 'beds' - категориальный, но можно сделать числовым, разделив признак на два признака: 'count_beds' и 'sqft_beds'

In [326]:
df['sqft_beds'] = df['beds'].apply(lambda x: x if -1!=x.find('sqft') else '0')

df['sqft_beds'].value_counts()

In [327]:
df['sqft_beds'] = df['beds'].apply(lambda x: float(re.sub('[^0-9\.]', '', x).strip())
                                   if -1!=x.find('sqft') else 0)

In [328]:
df['sqft_beds'].value_counts()

In [329]:
df['count_beds'] = df['beds'].apply(lambda x: x if (-1==x.find('sqft'))  else '0')

In [330]:
df['count_beds'].value_counts()

In [331]:
df['beds'] = df['beds'].apply(lambda x: '999 Beds' if -1!=x.find('Bath') else x)
df['count_beds'] = df['beds'].apply(lambda x: float(re.sub('[^0-9\.]', '', x).strip())
                                   if -1!=x.find('Beds') else 0)

In [332]:
df['count_beds'].value_counts()

Для признака  поставим значение 999 (большое число) вместо 'Bath' для того, чтобы подчеркнуть значимость совмещенных спальных комнат с ванной.

In [333]:
df.info()

In [334]:
#  Удаляется признаки: вспомогательные, рабочие и расшипленные на отдельные признаки
feature_drop = ['schools',  'homeFacts', 'beds']
df.drop(feature_drop, axis=1, inplace=True)

In [335]:
df.info()

Признак "target" надо преобразовать float

# Результаты EDA

Датасет очень "грязный". Цель -  максимально сохранить исходные данные. Поэтому пришлось изменить и исправить даже одиночные строки.
* Слишком длительное время было уделено для чистки датасет.
* В датасет очень большое число категориальных признаков. На следующем этапе, при выборе методов прогноза необходимо учитывать эту особенность проекта.
* Следующим ответственным этапом будет подборка важных признаков и их типа, которые определяются с помощью модельных экспериментов. 
* Выбор модели для прогноза особой сложности не представляет.
* Для прогнозных расчетов используется Kaggle.

Окончательный результат постараюсь оформить в виде презентации.